In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

# Load dataset
df = pd.read_csv("emergency_patient_dataset.csv")

# Check the actual columns in the dataset
print(df.columns)

# Encode categorical labels
label_encoders = {}
categorical_columns = ["Predicted Disease", "Triage Level", "Suggested Treatment", "Consciousness Level", "Symptoms"]

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save for later decoding

# Save label encoders
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

# Drop unnecessary features if they exist
columns_to_drop = ["Predicted Disease", "Triage Level", "Suggested Treatment", "Gender", "Previous Conditions"]
columns_in_df = [col for col in columns_to_drop if col in df.columns]
X = df.drop(columns=columns_in_df)

y_disease = df["Predicted Disease"]
y_triage = df["Triage Level"]
y_treatment = df["Suggested Treatment"]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Split dataset into training and validation sets
X_train, X_test, y_disease_train, y_disease_test, y_triage_train, y_triage_test, y_treatment_train, y_treatment_test = train_test_split(
    X_scaled, y_disease, y_triage, y_treatment, test_size=0.2, random_state=42
)

# Model input
input_layer = Input(shape=(X_train.shape[1],))
shared = Dense(128, activation='relu')(input_layer)
shared = Dropout(0.3)(shared)
shared = Dense(64, activation='relu')(shared)

# Output layers for multi-task learning
disease_output = Dense(len(df["Predicted Disease"].unique()), activation='softmax', name='disease_output')(shared)
triage_output = Dense(len(df["Triage Level"].unique()), activation='softmax', name='triage_output')(shared)
treatment_output = Dense(len(df["Suggested Treatment"].unique()), activation='softmax', name='treatment_output')(shared)

# Create the model
model = Model(inputs=input_layer, outputs=[disease_output, triage_output, treatment_output])
model.compile(optimizer='adam',
              loss=['sparse_categorical_crossentropy'] * 3,
              metrics=[['accuracy'], ['accuracy'], ['accuracy']])

# Add model checkpointing to save the best model
checkpoint = keras.callbacks.ModelCheckpoint("multi_output_model.h5", monitor="val_loss", save_best_only=True, verbose=1)

# Train the model
history = model.fit(X_train, [y_disease_train, y_triage_train, y_treatment_train],
                    validation_data=(X_test, [y_disease_test, y_triage_test, y_treatment_test]),
                    epochs=30, batch_size=32, callbacks=[checkpoint])

# Evaluate the model
eval_results = model.evaluate(X_test, [y_disease_test, y_triage_test, y_treatment_test])
print(f"Disease Prediction Accuracy: {eval_results[1]:.4f}")
print(f"Triage Level Prediction Accuracy: {eval_results[2]:.4f}")
print(f"Treatment Suggestion Accuracy: {eval_results[3]:.4f}")

Index(['Heart Rate (bpm)', 'Oxygen Saturation (%)',
       'Respiratory Rate (breaths per min)', 'Systolic Blood Pressure (mmHg)',
       'Diastolic Blood Pressure (mmHg)',
       'ECG Abnormality (0 = Normal, 1 = Abnormal)',
       'X-ray Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue)',
       'CT Scan Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue)',
       'Age', 'Temperature (°F)', 'Pain Level (1-10)', 'Consciousness Level',
       'Symptoms', 'Predicted Disease', 'Triage Level', 'Suggested Treatment'],
      dtype='object')
Epoch 1/30
483/500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - disease_output_accuracy: 0.1422 - disease_output_loss: 1.9645 - loss: 4.8584 - treatment_output_accuracy: 0.1683 - treatment_output_loss: 1.8172 - triage_output_accuracy: 0.3993 - triage_output_loss: 1.0767
Epoch 1: val_loss improved from inf to 4.79665, saving model to multi_output_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - disease_output_accuracy: 0.1422 - disease_output_loss: 1.9643 - loss: 4.8573 - treatment_output_accuracy: 0.1682 - treatment_output_loss: 1.8168 - triage_output_accuracy: 0.3995 - triage_output_loss: 1.0763 - val_disease_output_accuracy: 0.1447 - val_disease_output_loss: 1.9483 - val_loss: 4.7966 - val_treatment_output_accuracy: 0.1692 - val_treatment_output_loss: 1.7951 - val_triage_output_accuracy: 0.4095 - val_triage_output_loss: 1.0532
Epoch 2/30
495/500 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - disease_output_accuracy: 0.1469 - disease_output_loss: 1.9476 - loss: 4.7992 - treatment_output_accuracy: 0.1703 - treatment_output_loss: 1.7943 - triage_output_accuracy: 0.4048 - triage_output_loss: 1.0574
Epoch 2: val_loss improved from 4.79665 to 4.79199, saving model to multi_output_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - disease_output_accuracy: 0.1469 - disease_output_loss: 1.9476 - loss: 4.7992 - treatment_output_accuracy: 0.1703 - treatment_output_loss: 1.7942 - triage_output_accuracy: 0.4048 - triage_output_loss: 1.0574 - val_disease_output_accuracy: 0.1435 - val_disease_output_loss: 1.9482 - val_loss: 4.7920 - val_treatment_output_accuracy: 0.1595 - val_treatment_output_loss: 1.7964 - val_triage_output_accuracy: 0.4178 - val_triage_output_loss: 1.0474
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - disease_output_accuracy: 0.1556 - disease_output_loss: 1.9441 - loss: 4.7881 - treatment_output_accuracy: 0.1807 - treatment_output_loss: 1.7900 - triage_output_accuracy: 0.4153 - triage_output_loss: 1.0540
Epoch 3: val_loss improved from 4.79199 to 4.79176, saving model to multi_output_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - disease_output_accuracy: 0.1556 - disease_output_loss: 1.9441 - loss: 4.7881 - treatment_output_accuracy: 0.1807 - treatment_output_loss: 1.7900 - triage_output_accuracy: 0.4153 - triage_output_loss: 1.0540 - val_disease_output_accuracy: 0.1482 - val_disease_output_loss: 1.9470 - val_loss: 4.7918 - val_treatment_output_accuracy: 0.1593 - val_treatment_output_loss: 1.7954 - val_triage_output_accuracy: 0.4072 - val_triage_output_loss: 1.0493
Epoch 4/30
490/500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - disease_output_accuracy: 0.1603 - disease_output_loss: 1.9416 - loss: 4.7887 - treatment_output_accuracy: 0.1787 - treatment_output_loss: 1.7898 - triage_output_accuracy: 0.4100 - triage_output_loss: 1.0573
Epoch 4: val_loss improved from 4.79176 to 4.79112, saving model to multi_output_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - disease_output_accuracy: 0.1604 - disease_output_loss: 1.9417 - loss: 4.7887 - treatment_output_accuracy: 0.1787 - treatment_output_loss: 1.7898 - triage_output_accuracy: 0.4101 - triage_output_loss: 1.0572 - val_disease_output_accuracy: 0.1497 - val_disease_output_loss: 1.9482 - val_loss: 4.7911 - val_treatment_output_accuracy: 0.1692 - val_treatment_output_loss: 1.7956 - val_triage_output_accuracy: 0.4200 - val_triage_output_loss: 1.0474
Epoch 5/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - disease_output_accuracy: 0.1615 - disease_output_loss: 1.9411 - loss: 4.7806 - treatment_output_accuracy: 0.1847 - treatment_output_loss: 1.7885 - triage_output_accuracy: 0.4156 - triage_output_loss: 1.0510
Epoch 5: val_loss did not improve from 4.79112
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - disease_output_accuracy: 0.1615 - disease_output_loss: 1.9411 - loss: 4.7807 - treatment_output_accuracy: 0.1847 - treatment_output_loss: 1.7885 - triage_output_a

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle
from sklearn.preprocessing import StandardScaler

# Load saved model
model = keras.models.load_model("multi_output_model.h5")

# Load label encoders
with open("label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)

# Load scaler
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# Check expected number of features
expected_features = scaler.n_features_in_
print(f"Scaler expects {expected_features} features.")  # Debugging statement

# Function to take user input (till pain level)
# Function to take user input (including missing features)
# Function to take user input (including Consciousness Level and Symptoms)
def get_user_input():
    print("\nEnter the patient's details:")
    try:
        heart_rate = float(input("Heart Rate (bpm): "))
        oxygen_saturation = float(input("Oxygen Saturation (%): "))
        respiratory_rate = float(input("Respiratory Rate (breaths per min): "))
        systolic_bp = float(input("Systolic Blood Pressure (mmHg): "))
        diastolic_bp = float(input("Diastolic Blood Pressure (mmHg): "))
        ecg_abnormality = int(input("ECG Abnormality (0 = Normal, 1 = Abnormal): "))
        xray_findings = int(input("X-ray Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue): "))
        ct_findings = int(input("CT Scan Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue): "))
        age = int(input("Age: "))
        temperature = float(input("Temperature (°F): "))
        pain_level = int(input("Pain Level (1-10): "))

        # Add missing fields
        consciousness_level = int(input("Consciousness Level (0 = Unconscious, 1 = Conscious): "))

        # For Symptoms, we'll assume the user provides a categorical value (0 = None, 1 = Mild, 2 = Severe)
        symptoms = int(input("Symptoms (0 = None, 1 = Mild, 2 = Severe): "))

        # Create input array (13 features)
        input_data = np.array([[heart_rate, oxygen_saturation, respiratory_rate, systolic_bp, diastolic_bp,
                                ecg_abnormality, xray_findings, ct_findings, age, temperature, pain_level,
                                consciousness_level, symptoms]])

        # Check input shape
        print(f"User input shape: {input_data.shape}")  # Debugging statement

        # Normalize input
        input_scaled = scaler.transform(input_data)
        return input_scaled

    except ValueError as e:
        print(f"Invalid input: {e}")
        return None

# Function to make predictions
def predict_patient(input_data):
    if input_data is None:
        return None  # Return if input is invalid

    predictions = model.predict(input_data)

    disease_pred = label_encoders["Predicted Disease"].inverse_transform([np.argmax(predictions[0])])[0]
    triage_pred = label_encoders["Triage Level"].inverse_transform([np.argmax(predictions[1])])[0]
    treatment_pred = label_encoders["Suggested Treatment"].inverse_transform([np.argmax(predictions[2])])[0]

    return {
        "Predicted Disease": disease_pred,
        "Triage Level": triage_pred,
        "Suggested Treatment": treatment_pred
    }

# Get user input and make prediction
user_input = get_user_input()
if user_input is not None:
    prediction = predict_patient(user_input)

    if prediction:
        print("\nPrediction Results:")
        print(f"Predicted Disease: {prediction['Predicted Disease']}")
        print(f"Triage Level: {prediction['Triage Level']}")
        print(f"Suggested Treatment: {prediction['Suggested Treatment']}")

Scaler expects 13 features.

Enter the patient's details:
Heart Rate (bpm): 107
Oxygen Saturation (%): 62
Respiratory Rate (breaths per min): 32
Systolic Blood Pressure (mmHg): 181
Diastolic Blood Pressure (mmHg): 89
ECG Abnormality (0 = Normal, 1 = Abnormal): 0
X-ray Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue): 2
CT Scan Findings (0 = Normal, 1 = Minor Issue, 2 = Critical Issue): 1
Age: 20
Temperature (°F): 98
Pain Level (1-10): 10
Consciousness Level (0 = Unconscious, 1 = Conscious): 1
Symptoms (0 = None, 1 = Mild, 2 = Severe): 1
User input shape: (1, 13)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step

Prediction Results:
Predicted Disease: Sepsis
Triage Level: High
Suggested Treatment: Painkillers - 500mg every 4 hours
